In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os

os.chdir("/content/drive/My Drive/Colab/assignment 2")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
myDataFrame2 = pd.read_csv('refugee.csv')
myDataFrame2.head()

,AgreementId,Version,Name,Region,Country,Peace Process,Peace Process Name,Stage,Signed Date,Agreement/Conflict Level,Conflict Nature,Agreement Status,Agreement Text
0,1739,2,Agreement between the Islamic Republic of Afgh...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,SubPar,2016-09-22,Intra,Government,Multiparty signed/agreed,Agreement Between Goira & Hezb-E-Islami Of Afg...
1,864,1,Tokyo Declaration Partnership for Self-Relianc...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2012-07-08,InterIntra,Government,Multiparty signed/agreed,The Tokyo Declaration Partnership for Self-Rel...
2,848,1,Conclusions of the Conference on Afghanistan a...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Pre,2011-12-05,InterIntra,Government,Multiparty signed/agreed,The International Afghanistan Conference in Bo...
3,849,1,Istanbul Process on Regional Security and Coop...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2011-11-02,InterIntra,Government,Multiparty signed/agreed,Istanbul Process on Regional Security and Coop...
4,709,1,Renewed Commitment by the Afghan Government to...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2010-07-22,InterIntra,Government,Multiparty signed/agreed,A Renewed Commitment by the Afghan Government ...


### Agreement Text
*   The last one to describe is the **Agreement Text** within this data. There are many stopwords need to be removed, and keywords need to be extracted.

In [11]:
myDataFrame5=myDataFrame2[['Agreement Text']]
myDataFrame5

,Agreement Text
0,Agreement Between Goira & Hezb-E-Islami Of Afg...
1,The Tokyo Declaration Partnership for Self-Rel...
2,The International Afghanistan Conference in Bo...
3,Istanbul Process on Regional Security and Coop...
4,A Renewed Commitment by the Afghan Government ...
...,...
485,APPENDIX F\r\nConflict Ceasefire Conditions\r\...
486,Annex\r\n\r\nAGREEMENT ON A CEASE-FIRE IN THE ...
487,\r\n\r\nPreamble \r\n\r\nWe the people of Zim...
488,Agreement between ZANU-PF and the two MDC form...


Create a new file for future use.

In [12]:
myDataFrame5.to_csv('agreement.csv')

Delete all blank lines in the file and save as another file.

In [13]:
f=open('agreement.csv')
g=open('re_agreement.csv','w')
try:
    while True:
        line=f.readline()
        if len(line)==0:
            break
        if line.count('\n')==len(line):
            continue
        g.write(line)
finally:
    f.close()
    g.close()

Clean the text with spacy method.

In [14]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    ldatokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            ldatokens.append('URL') 
        elif token.orth_.startswith('@'):
            ldatokens.append('SCREEN_NAME')
        else:
            ldatokens.append(token.lower_)
    return ldatokens

Lemmatization with wordnet method.

In [15]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def lemma1(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
from nltk.stem.wordnet import WordNetLemmatizer
def lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Use nltk to remove stopwords.

In [16]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
  def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens]
    return tokens

# lda

This is the data prepared for lda model, which can also first show some high frenquency words in the text.

In [18]:
import random
text_data = []
with open('re_agreement.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['goira', 'pledge', 'recruit', 'eligible', 'individual', 'commander', 'interest', 'serve', 'country', 'security', 'defense', 'force', 'goira', 'commit', 'prepare', 'ground', 'legal', 'honorable', 'reliable', 'integration', 'society', 'knowhow', 'implementation', 'article', 'arrange', 'joint', 'executive', 'commission']
['article', 'seventeenth']
['article', 'twenty', 'second']
['enact', 'enforce', 'legal', 'framework', 'fighting', 'corruption', 'include', 'example', 'annual', 'asset', 'declaration', 'senior', 'public', 'official', 'include', 'executive', 'legislative', 'judiciary']
['consistent', 'transition', 'reaffirm', 'international', 'actor', 'evolve', 'direct', 'service', 'delivery', 'support', 'capacity‐building', 'afghan', 'institution', 'enable', 'government', 'afghanistan', 'exercise', 'sovereign', 'authority', 'function', 'process', 'include', 'phase', 'provincial', 'reconstruction', 'team', 'dissolution', 'structure', 'duplicate', 'function', 'authority', 'government', 'afg

Use for building matrix.

In [19]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

5 topics for lda model.

In [20]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.036*"shall" + 0.011*"committee" + 0.009*"state" + 0.008*"community"')
(1, '0.020*"agreement" + 0.017*"party" + 0.011*"peace" + 0.011*"government"')
(2, '0.022*"shall" + 0.014*"national" + 0.009*"development" + 0.007*"right"')
(3, '0.033*"shall" + 0.014*"government" + 0.012*"commission" + 0.010*"president"')
(4, '0.018*"article" + 0.018*"national" + 0.017*"shall" + 0.012*"right"')


3 topics for lda model.

In [21]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.039*"shall" + 0.010*"agreement" + 0.008*"government" + 0.008*"force"')
(1, '0.013*"article" + 0.009*"republic" + 0.008*"right" + 0.007*"national"')
(2, '0.019*"national" + 0.018*"shall" + 0.012*"state" + 0.010*"government"')


10 topics for lda model.

In [22]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.024*"shall" + 0.016*"commission" + 0.014*"party" + 0.011*"international"')
(1, '0.023*"shall" + 0.009*"member" + 0.009*"election" + 0.008*"palestinian"')
(2, '0.059*"shall" + 0.015*"national" + 0.011*"government" + 0.009*"member"')
(3, '0.017*"shall" + 0.013*"military" + 0.013*"security" + 0.011*"national"')
(4, '0.029*"agreement" + 0.016*"shall" + 0.013*"government" + 0.013*"peace"')
(5, '0.014*"shall" + 0.009*"government" + 0.009*"president" + 0.008*"republic"')
(6, '0.027*"shall" + 0.014*"state" + 0.013*"committee" + 0.010*"government"')
(7, '0.016*"shall" + 0.016*"government" + 0.016*"conflict" + 0.013*"president"')
(8, '0.049*"article" + 0.012*"court" + 0.012*"party" + 0.010*"right"')
(9, '0.012*"shall" + 0.011*"ensure" + 0.011*"party" + 0.010*"government"')


## Explore

### Use pyldavis to visualize different frequent words under different topics.

In [23]:
!pip install pyldavis

     |████████████████████████████████| 1.6MB 20.9MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=40d67f2f62ae9bde3cb4193604a1cd48255e88149cac92ee460c704eef5338d3
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


难民 5 主题

In [24]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

难民 3 主题

In [26]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


难民 10 主题

In [27]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
